# Test your AI component compatibility

This notebook show how your AI component can be built and then tested, to know if it will be compatible with the pipeline evaluation process.

This notebook give an overview about how an AI component candidate will be evaluated.  Even if it does not compute metrics, it give a precise idea , about how to build an compatible AI component, and how it is used in inference for score generation.

## Objective

The task is to build a AI component whose main task whose main objective is to predict the welding state from a given image.

### Inputs to the AI component
The AI component shall takes as input : 
- A list of numpy arrays representing the list of input images to process . 
- A list of dictionnary containing a meta-description of the image.

### The outputs of the AI component  
It shall return a dictionnary with four keys {predictions , probabilities, OOD_score, explainabilities}
    
First key is required    
- *predictions*:  The list of predicted welding state. The welding state can have three possible values: [OK, KO, UNKNOWN]
    
The following keys are not mandatory, but will greatly participate to the improvement of the quality score for the developed AI component if present:

- *probabilities*:  The list of associated probabilities for each images [$P_{KO}$, $P_{OK}$, $P_{UNKNWON}$]  where $\sum_{i \in \{\text{OK, KO, UNKNOWN}\}} P_i = 1$.

- *OOD_scores*: The list OOD score predicted by the AI component of for each images. This score X is a real positive. If $0\leq X < 1$ the image is considered as *In-Domain*, if $X >1$ the image is considered as *Out-of-Domain (OoD)*.

- *explainabilities*: The list of explainabilities for each input images. An explainability is an intensity matrix ( matrix with values between 0 and 1) with same size of the image tensor, that represents the importance of each pixel in the model prediction






### Prerequisites
Install the dependencies if it is not already done. For more information look at the [readme](../README.md) file.

##### For development on Local Machine

In [ ]:
### Install a virtual environment
# Option 1:  using conda (recommended)
# !conda create -n venv python=3.12
# !conda activate venv
# !pip install torch==2.6.0

# Option 2: using virtualenv
# !pip install virtualenv
# !virtualenv -p /usr/bin/python3.12 venv
# !source venv_lips/bin/activate

### Install the welding challenge package
# Option 1: Get the last version from Pypi
# !pip install 'challenge_welding'

# Option 2: Get the last version from github repository
# !git clone https://github.com/XX
# !pip install -U .

##### For Google Colab Users
You could also use a GPU device from Runtime > Change runtime type and by selecting T4 GPU.

In [ ]:
### Install the welding challenge package
# Option 1: Get the last version of LIPS framework from PyPI (Recommended)
# !pip install 'XX'
# !pip install torch==2.6.0

In [ ]:
# Option 2: Get the last version from github repository
# !git clone https://github.com/XX
# !pip install -U .
# !pip install torch==2.6.0

Attention: You may restart the session after this installation, in order that the changes be effective.

In [ ]:
# Clone the starting kit
# !git clone https://github.com/confianceai/Challenge-Welding-Starter-Kit.git
# and change the directory to the starting kit to be able to run correctly this notebook
# import os
# os.chdir("Challenge-Welding-Starter-Kit")

## Build your Ai component

An AI component shall be a buildable python package . Thus, it is a folder that shall have at least the following files and folders :
   ```
    /
    MANIFEST.in
    setup.py
    requirements.txt
    challenge_solution/
        AIcomponent.py
        __init__.py
 ```

You can see [Requirements and evaluation process](../docs/Requirements_and_Evaluation_process.md)

Only those files will be used by the evaluation pipeline to test your AI component. The names of files and folders shall not be changed.
The most important file is the AIcomponent.py file that is the interface of your AI component. Only this interface will be used by the evaluation pipeline to interact with your component. That is why this file require some strict named methods and class to be present. It shall follow this abstract class [AIComponent interface](../challenge_welding/AIComponent_interface.py)

You are free to add other files to make your component working.

The easiest way to build your AI component for the challenge is to start from the AI component template given in the ```AIcomponent_template``` folder of this repository and complete this folder following the process described in its readme.md .

## Test an AI component compatibility

In this section, we will test the AI component compatibility with the evaluation pipeline of this challenge. The following lines will test if the proposed AI component can be loaded properly in the evaluation pipeline, and used for inference computation on a given dataset.
We does not provide the computations metrics function here. But all score metrics computation functions used by the evaluation pipeline are based only on the inference results of the AI component on many evaluation datasets. Thus if the inference works on your AI component , it will ensure that the score computation and the full evaluation process will work too .

In this example, we use for the test the reference solution provided within this challenge and accessible here : 

In [ ]:
# Set here the path of the AI component to test here, you can set a local filsystem path, or an url to a public git repository
# You can replace it by the path to your own component to test

# AI_component_path= "../../Challenge-Welding-Reference-Solution-1"  
AI_component_path="https://github.com/confianceai/Challenge-Welding-Reference-Solution-1"

In [ ]:
import sys
# sys.path.insert(0, "..") # Uncomment this line For local tests without pkg installation, to make challenge_welding module visible 
from challenge_welding.user_interface import ChallengeUI
from challenge_welding.inference_tools import TestAIComponent

## Launch the test pipeline

The test pipeline take an AI component (the solution to test) and perform the following tasks

- Install your AI component as a python package 
- Load the AI component of the solution you want to test
- Apply inference on this AI component on one or many evaluation datasets. Each inference process on a dataset generate as output a dataframe( stored as a parquet file) containing evaluation dataset metadata extended with prediction results.


## Init the pipeline

In [ ]:
# Initialize test pipeline
myPipeline=TestAIComponent(proposed_solution_path=AI_component_path, # Set here the AI component path you want to evaluate
                              meta_root_path="starter_kit_test_AI_comp_results", # Set the directory here where pipeline results will be stored (inference results, and computed metrics)
                              cache_strategy="local", # "local" or "remote" .If set on "local", all image used for evaluation , will be locally stored in a cache directory. Else, image will be loaded directly from downloding
                              cache_dir="test_cache") # chosen directory for cache

## Load your AI component into the evaluation environnement

The load_proposed_solution() method below is divided into two main tasks:
- Install the python package of your AI component --> ( execute the commande pip install AI_comp_path)
- Call the load_model() method of your AIcomponent interface


In [ ]:
myPipeline.load_proposed_solution()

current_dir C:\SAUVEGARDES_FIN_CONFIANCE\CSIA++\Challenge\Build2025\env-test-v2-starter-kit\Lib\site-packages\challenge_solution
AI component loaded


C:\SAUVEGARDES_FIN_CONFIANCE\CSIA++\Challenge\Build2025\env-test-v2-starter-kit\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Load an evaluation dataset metadescription

In the next cell, we load the metadata of the evaluation dataset we want to use to evaluate our AI component

In [ ]:
# In this example we will choose a small dataset

ds_name="example_mini_dataset"

# Load all metadata of your dataset as a pandas dataframe, (you can point to a local cache metafile instead of original one pointing on remote repository)

my_challenge_UI=ChallengeUI()
evaluation_ds_meta_df=my_challenge_UI.get_ds_metadata_dataframe(ds_name)

display(evaluation_ds_meta_df.head(5))

https://minio-storage.apps.confianceai-public.irtsysx.fr/challenge-welding/datasets/example_mini_dataset/metadata/ds_meta.parquet


,sample_id,class,timestamp,welding-seams,labelling_type,resolution,path,sha256,storage_type,data_origin,blur_level,blur_class,luminosity_level,external_path
0,data_92409,OK,22/01/20 12:49,c33,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'GN\xd7\xa7B\x98\xb0r\xa4\xdfn\x8cT\x8e:\xc07...,s3,real,701.938341,blur,50.533365,http://minio-storage.apps.confianceai-public.i...
1,data_67943,OK,20/02/20 23:53,c102,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b's\xf6;3i-\x10\xfd8y\xf2\xe1\xa6JQ\x84`\xc6\x...,s3,real,715.670702,blur,47.050604,http://minio-storage.apps.confianceai-public.i...
2,data_4843,OK,20/01/20 20:34,c20,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'\xdbZ\xb3\x12e&\xd5\x83\x13*\x87S\xe1\x19\xc...,s3,real,715.857380,blur,46.204245,http://minio-storage.apps.confianceai-public.i...
3,data_25309,OK,18/07/2022 20:18,c102,operator,"[960, 540]",challenge-welding/datasets/example_mini_datase...,b'/c\xe3\xd9\xc8|&\xaf\xb1}\xf6\xe3s\xae\xea\x...,s3,real,869.513006,blur,34.359280,http://minio-storage.apps.confianceai-public.i...
4,data_76144,OK,03/10/19 21:14,c20,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'\xca%\x0c\x92\x1f\x0c\x00\xcc\x02\r\xb8\xf1\...,s3,real,2676.246904,clean,46.256244,http://minio-storage.apps.confianceai-public.i...


##  Perform inference on an evaluation dataset

We pass the evaluation_dataframe in the method below. It use the loaded AI component to perform inference of each sample referenced in the evaluation dataframe and add the inference results as new columns

In [ ]:
result_df=myPipeline.perform_grouped_inference(evaluation_dataset=evaluation_ds_meta_df, # dataframe containing metadescription of your evaluation ds
                                               results_inference_path=myPipeline.meta_root_path+"/res_inference.parquet", # path to file that will contains inference_results
                                               batch_size=150 # You can group inference by batch if you want
                                              ) 

Number of  batch to process for inference :  20  , start processing..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:33<00:00,  1.70s/it]

cumulated inference time  33.964887857437134


You can see the inference results below. See that new column has been added :
- "predicted_state" imported from "predicitions" key of your AI component predict method output dictionnary
- "scores KO" imported from "probabilities" key of your AI component predict method output dictionnary
- "scores OK" imported from "probabilities" key of your AI component predict method output dictionnary
- "score OOD" imported from "OOD scores" key of your AI component predict method output dictionnary


In [ ]:
display(result_df)

,class,timestamp,welding-seams,labelling_type,resolution,path,sha256,storage_type,data_origin,blur_level,blur_class,luminosity_level,external_path,predicted_state,scores KO,scores OK,scores UNKNOWN,scores OOD,compute_time
sample_id,,,,,,,,,,,,,,,,,,,
data_92409,OK,22/01/20 12:49,c33,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'GN\xd7\xa7B\x98\xb0r\xa4\xdfn\x8cT\x8e:\xc07...,s3,real,701.938341,blur,50.533365,http://minio-storage.apps.confianceai-public.i...,OK,0.276092,0.723908,0.0,0.939183,0.004477
data_67943,OK,20/02/20 23:53,c102,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b's\xf6;3i-\x10\xfd8y\xf2\xe1\xa6JQ\x84`\xc6\x...,s3,real,715.670702,blur,47.050604,http://minio-storage.apps.confianceai-public.i...,UNKNOWN,0.000011,0.499989,0.5,1.307583,0.004477
data_4843,OK,20/01/20 20:34,c20,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'\xdbZ\xb3\x12e&\xd5\x83\x13*\x87S\xe1\x19\xc...,s3,real,715.857380,blur,46.204245,http://minio-storage.apps.confianceai-public.i...,OK,0.194817,0.805183,0.0,0.710907,0.004477
data_25309,OK,18/07/2022 20:18,c102,operator,"[960, 540]",challenge-welding/datasets/example_mini_datase...,b'/c\xe3\xd9\xc8|&\xaf\xb1}\xf6\xe3s\xae\xea\x...,s3,real,869.513006,blur,34.359280,http://minio-storage.apps.confianceai-public.i...,UNKNOWN,0.001472,0.498528,0.5,1.182224,0.004477
data_76144,OK,03/10/19 21:14,c20,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'\xca%\x0c\x92\x1f\x0c\x00\xcc\x02\r\xb8\xf1\...,s3,real,2676.246904,clean,46.256244,http://minio-storage.apps.confianceai-public.i...,OK,0.022103,0.977897,0.0,0.905816,0.004477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
data_41485,OK,26/07/2022 00:51,c33,operator,"[960, 540]",challenge-welding/datasets/example_mini_datase...,b'\tK\x04\xf1h\x19:\xcd\xe1\xbc\xe6\xd2\xb6\x0...,s3,real,2158.350896,clean,46.735307,http://minio-storage.apps.confianceai-public.i...,UNKNOWN,0.010414,0.489586,0.5,1.069381,0.009127
data_79596,OK,15/07/20 08:27,c20,expert,"[1920, 1080]",challenge-welding/datasets/example_mini_datase...,b'\xf0\xd9\x9f\x16]\xacXY\xb5\x06rpv/\xbf\xc2\...,s3,real,2314.094661,clean,47.251992,http://minio-storage.apps.confianceai-public.i...,UNKNOWN,0.202769,0.297231,0.5,1.026727,0.009127
data_26044,OK,21/07/2022 14:25,c102,operator,"[960, 540]",challenge-welding/datasets/example_mini_datase...,b'\x8c\r\x83\xeb\xf8\x93\xd9\x1a\xda\xea\xa8*\...,s3,real,976.539956,clean,41.172945,http://minio-storage.apps.confianceai-public.i...,UNKNOWN,0.016342,0.483658,0.5,1.021310,0.009127


Check the output dataframe contain columns corresponding to ouptut fields filled with correct values. 
Here the reference component tested shall create columns named, 
- predicted_states
- scores_KO
-  scores OK
-   OOD_scores
-   
If your result dataframe is correct and the parquet is well created, then your AI comp is compatible with the evaluation pipeline


# Evaluation process

From the predictions made by the developed AI component on many evaluation datasets, we will compute a set of different evaluation criteria as discussed below:

- **Operationnal metrics**: Measure the gain brought the AI component compared to a human only qualification process. This metrics is based on the confusion matrix and penalize strongly false negative predictions. 

- **Uncertainty metrics**: Measure the ability of the AI component to produce a calibrated prediction confidence indicator expressing risk of model errors.

- **Robustness metrics**: Measure the ability of the AI component to have invariant output is images have slight perturbations (blut, luminosity, rotation, translation)

- **Monitoring metrics**: Measure the ability of the AI component to detect if an input image is ood, and gives the appropriate output ->Unknown

- **Explainability metrics**: Measure the ability of the AI component to give appropriate explanations

Add here tables containing each metrics by themes (operationnal, uncertainty, robustness, monitoring, explainabilities) and  complementary text to detail each unit metrics

# Scoring

Explain here how metrics are aggregated to build final score 